In [9]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv("extracted_quotes.csv", index_col = None)

In [3]:
df.head(5)

,quote,source,author,heading_context
0,Un homme heureux est trop content du présent p...,A happy man is too satisfied with the present ...,Albert Einstein,1890s
1,Autoritätsdusel ist der größte Feind der Wahrh...,Blind obedience to authority is the greatest e...,Albert Einstein,1900s
2,Lieber Habicht! / Es herrscht ein weihevolles ...,"Dear Habicht, / Such a solemn air of silence h...",Albert Einstein,1900s
3,E=mc²,The equation originally expressed the equivale...,Albert Einstein,1900s
4,The mass of a body is a measure of its energy ...,Ist die Trägheit eines Körpers von seinem Ener...,Albert Einstein,1900s


Let's perform some exploratory dataset analysis on the dataset

In [4]:
df.shape

(30000, 4)

In [15]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   quote            29984 non-null  object
 1   source           22914 non-null  object
 2   author           30000 non-null  object
 3   heading_context  20671 non-null  object
dtypes: object(4)
memory usage: 937.6+ KB
None


In [17]:
# Missing values
print(df.isna().sum())

quote                16
source             7086
author                0
heading_context    9329
dtype: int64


There are some quotes which are null lets see those.

In [18]:
null_quotes = df[df['quote'].isna() | df['quote'].str.strip().eq('')]

print(f"Number of null/empty quotes: {len(null_quotes)}")
display(null_quotes[['quote', 'author', 'source', 'heading_context']].head(20))

Number of null/empty quotes: 16


,quote,author,source,heading_context
13703,NaN,Chinese proverbs,Transliteration (pinyin): Chángjiāng hòulàng t...,Ch
13704,NaN,Chinese proverbs,Transliteration (pinyin): Dú wàn juǎn shū bùrú...,D
13708,NaN,Chinese proverbs,Transliteration (pinyin): Fù zhài zǐ huán. Tra...,F
13709,NaN,Chinese proverbs,Transliteration (pinyin): Hài rén zhī xīn bù k...,H
13712,NaN,Chinese proverbs,"Transliteration (pinyin): Kōngxuéláifēng, wèib...",K
13713,NaN,Chinese proverbs,Transliteration (pinyin): Liángyào kǔkǒu Tradi...,L
13714,NaN,Chinese proverbs,Transliteration: Yǒu qí fù bì yǒu qí zǐ. Havin...,L
13715,NaN,Chinese proverbs,Transliteration (pinyin): Rén suàn bùrú tiān s...,R
13717,NaN,Chinese proverbs,Transliteration (pinyin): Sān gè héshàng méi s...,S
13718,NaN,Chinese proverbs,Transliteration (pinyin): Sǐ mǎ dāng huó mǎ yī...,S


The probelm is that the chinese proverbs are in chinese language and while extracting the quotes we only focused on the english language characters that's why they are null.

Let's see if all the chinese proverbs are like this

In [27]:
chinese_quotes = df[df['author'].str.strip().eq('Chinese proverbs')]

print(f"Number of chinese quotes: {len(chinese_quotes)}")
display(chinese_quotes[['quote', 'author', 'source', 'heading_context']].head(20))

Number of chinese quotes: 61


,quote,author,source,heading_context
13699,， ， ， ； 。,Chinese proverbs,Transliteration (pinyin): Bù wén bù ruò wén zh...,B
13700,小洞不补，大洞吃苦,Chinese proverbs,"Transliteration: xiǎo dòng bù bǔ, dà dòng chī ...",B
13701,读书须用意，一字值千金,Chinese proverbs,"Transliteration: dú shū xū yòng yì, yī zì zhí ...",B
13702,宝剑锋从磨砺出，梅花香自苦寒来,Chinese proverbs,"Transliteration: Bǎojiàn fēng cóng mólì chū, m...",B
13703,NaN,Chinese proverbs,Transliteration (pinyin): Chángjiāng hòulàng t...,Ch
13704,NaN,Chinese proverbs,Transliteration (pinyin): Dú wàn juǎn shū bùrú...,D
13705,， ， https: //learnchinesewithabdul. com/chines...,Chinese proverbs,"pinyin: fēng xiàng biàn shí, yǒu rén jìng qián...",F
13706,https: //archive. org/details/diversproverbsw0...,Chinese proverbs,Transliteration (pinyin): Fáng rén zhī xīn bùk...,F
13707,",",Chinese proverbs,"Transliteration (pinyin): Fú wú chóng zhì, huò...",F
13708,NaN,Chinese proverbs,Transliteration (pinyin): Fù zhài zǐ huán. Tra...,F


In [7]:
pd.set_option('display.max_colwidth', None)
print(df.loc[13716, ['quote', 'source']])

quote                                                                                                                                                                                                                                      一
source    Transliteration (pinyin): Ròu bāozi dǎ gǒu 一 qù bù huítóu. Traditional: 肉包子打狗一去不回頭 Simplified: 肉包子打狗一去不回头 To hit a dog with a meat-bun, so it leaves never turning around. Meaning: Punishment gives less incentive than a reward.
Name: 13716, dtype: object


In [41]:
# pd.set_option('display.max_colwidth', None)

df.loc[df['author'] == "Chinese proverbs", 'source']

13699                                                                                                                                                        Transliteration (pinyin): Bù wén bù ruò wén zhī, wén zhī bù ruò jiàn zhī, jiàn zhī bù ruò zhīzhī, zhīzhī bù ruò xíng zhī; xué zhìyú xíng zhī ér zhǐ yǐ. Traditional: 不聞不若聞之，聞之不若見之，見之不若知之，知之不若行之；學至於行之而止矣 Simplified: 不闻不若闻之，闻之不若见之，见之不若知之，知之不若行之；学至于行之而止矣 From Xun Zi (荀子 8. 儒效 23）.
13700    Transliteration: xiǎo dòng bù bǔ, dà dòng chī kǔ A small hole not mended in time will become a big hole much more difficult to mend. English equivalent: "What's past is prologue. " or "A stitch in time saves nine. " Meaning: Fix something while it can be fixed. Don't wait until it's too late to do so. "Destroy the seed of evil, or it will grow up to your ruin. " Aesop, "The Swallow and the Other Birds" (c. 6th century BC)
13701                                                                                     Transliteration: dú shū xū yòng yì, yī z

WHen the source has the chinese alphabet and link attached to it in the source the extractor has avoided it as html link. So it is not able to extract it. But when the source is in english it has successfully extracted it.

Okay, now I understand. Multiple quotes on the chinese proverb page are written as a link and those proverbs have not been extracted and the translation has been mistakenly extracted in the source. 

So we have two options
* to drop the columns for which the quote is NaN value.
* to switch the source and the quote value where the quote is NaN. Because the source contains the translation and in a way it is also a quote.
    we will decide in the preprocessing part

Let's see the quotes which don't have both author and source.

In [14]:
missing_author_source = df[
    (df['author'].isna() | df['author'].str.strip().eq('')) &
    (df['source'].isna() | df['source'].str.strip().eq(''))
]

print(f"Quotes without both author and source: {len(missing_author_source)}")

Quotes without both author and source: 0


There aren't any quotes which don't have both source and author, which is a good thing. Let's check for the null values of each column.

Let's see how many unique authors do we have

In [29]:
unique_authors = df['author'].nunique(dropna=True)

# List of unique authors (optional)
authors_list = df['author'].dropna().unique()

print(f"Number of unique authors: {unique_authors}")
print(authors_list[:20])  # show first 20 authors

Number of unique authors: 368
['Albert Einstein' 'Disputed with Albert Einstein'
 'Misattributed to Albert Einstein' 'unknown' 'Zen proverbs'
 'Wikiquote: Templates' 'Latin proverbs' 'Nero' 'Martin Luther King Jr.'
 'Disputed with Martin Luther King Jr.'
 'Misattributed to Martin Luther King Jr.' 'John Cage' 'Harry S. Truman'
 'Disputed with Harry S. Truman' 'Misattributed to Harry S. Truman'
 'Bertrand Russell' 'Disputed with Bertrand Russell'
 'Misattributed to Bertrand Russell' 'Oscar Wilde'
 'Disputed with Oscar Wilde']


We have 368 unique authors but there is one mistake while extracting these from dump. Albert einstien and misattributed to ALbert Einstien and disputed with albert Einstin are three different authors in the author list. Which is a entity fragmentation.

Let's see who are the authors with most quotes and the authors with the least quotes.

In [30]:
author_counts = df['author'].value_counts().reset_index()
author_counts.columns = ['author', 'quote_count']

# Sort descending (most quotes first)
most_quotes = author_counts.sort_values(by='quote_count', ascending=False)

# Sort ascending (least quotes first)
least_quotes = author_counts.sort_values(by='quote_count', ascending=True)

print("Authors with most quotes:")
print(most_quotes.head(10))  # top 10

print("\nAuthors with least quotes:")
print(least_quotes.head(10))  # bottom 10

Authors with most quotes:
                   author  quote_count
0                 unknown         5342
1              Last words         1051
2        English proverbs          897
3          George W. Bush          674
4       Winston Churchill          504
5     Ralph Waldo Emerson          486
6        Bertrand Russell          480
7  Martin Luther King Jr.          460
8         John F. Kennedy          454
9          Thomas Carlyle          412

Authors with least quotes:
                                       author  quote_count
367          Disputed with Edsger W. Dijkstra            1
337       Misattributed to Edsger W. Dijkstra            1
336             Misattributed to Helen Keller            1
335         Misattributed to William Congreve            1
334             Misattributed to Walt Whitman            1
333             Disputed with Harry S. Truman            1
332            Misattributed to Stephen Crane            1
331               Disputed with H. L. Mencken

The misattributed and disputed we can deal later but the Last words and English Proverbs we should have a look.

Let's see if the source of the English Proverbs and Last Words are meaningful. Because the author does not contain any meaningfull information

In [31]:
# For English proverbs
print("\nEnglish proverbs:")
print(df[df['author'] == "English proverbs"].head(20))



English proverbs:
                                                   quote  \
23386               Absence makes the heart grow fonder.   
23387                       Long absent, soon forgotten.   
23388         The absent are always in the wrong. (1640)   
23389  Accidents will happen in the best families. (1...   
23390                   Actions speak louder than words.   
23391  Admiration: our polite recognition of another ...   
23392            He who does not advance goes backwards.   
23393                Advice most needed is least heeded.   
23394          Advise none to marry or go to war. (1640)   
23395                             Advisers run no risks.   
23396        All is fair in love and war. (17th century)   
23397         All is well that ends well. (14th century)   
23398    America is God's melting-pot. (Israel Zangwill)   
23399  Good riding at two anchors, men have told, for...   
23400  Anger makes dull men witty, but it keeps them ...   
23401  Do not let sun

I can see that the Quotes are scrapped very beautifully but the sources are missing for some I don't know why. I have used the same code to scrape all but for some the source are missing but for some it is present.

Should I revisit the extraction code? For now I think I should not because we dont have any quote for which both author and source are missing. SO collectively they will provide some information about the quote. SO for now I am not going to chaneg the extraction code code

In [32]:
# For English proverbs
print("\nLast Words:")
print(df[df['author'] == "Last words"].head(20))



Last Words:
                                                   quote  \
14083                                        No comment.   
14084                                I did what I could.   
14085                                         Van Halen!   
14086  Come Lord Jesus, come quickly, finish in me th...   
14087  يام السرور التي صفت لي دون تكدير في مدة سلطاني...   
14088  من در حال رفتن هستم و شما مي خواهيد غذا بخورم؟...   
14089                         I don't know. [Attributed]   
14090  שלף חרבך ומותתני פן-יאמרו לי, אשה הרגתהו (Shel...   
14091  May the Most High God preserve thee from destr...   
14092                It's okay! Gun's not loaded. . see?   
14093  Ja, maar niet te veel. |first= Bert |year= 199...   
14094  Oh, yes; it is the glorious Fourth of July. It...   
14095           This is the last of Earth. I am content.   
14096  Principally, and first of all, I recommend my ...   
14097             See in what peace a Christian can die.   
14098                      

In [34]:
pd.set_option('display.max_colwidth', None)
print(df.loc[14088, ['quote', 'source']])

quote                                                                                                       من در حال رفتن هستم و شما مي خواهيد غذا بخورم؟ (Man dar hâl raftan hastam, va shomâ mey khâhid ghazâ békhoram? )
source    Translation: You wish Me to take some food, and I am going? Who: `Abdu'l-Bahá, son of Bahá'u'lláh and one of three central figures of the Bahá'i Faith. Note: Spoken when food was offered to him on his deathbed.
Name: 14088, dtype: object


From the chinese proverb source, last words and english proverb's sources it is confirmed that if the quote is in another language or has translation then the source contains everything.

It is upto us to whether to trim the source or keep it as it is in the datbase. It will depend on the impact it has on the size of the graph database.

Let's analyse the heading context, how many unique values it has.

In [37]:
unique_headings = df['heading_context'].nunique(dropna=True)

# List of unique authors (optional)
heading_context_list = df['heading_context'].dropna().unique()

print(f"Number of unique headings: {unique_headings}")
print(heading_context_list[:20])  # show first 20 authors

Number of unique headings: 1724
['1890s' '1900s' '1910s' 'Principles of Research (1918)' '1920s'
 'Sidelights on Relativity (1922)' 'Viereck interview (1929)' '1930s'
 'Wisehart interview (1930)' 'Religion and Science (1930)'
 'What I Believe (1930)' 'Mein Weltbild (My World-view) (1931)'
 'My Credo (1932)' '1933' '1934' 'Obituary for Emmy Noether (1935)'
 'Why Do They Hate the Jews (1938)' '1940s' 'Science and Religion (1941)'
 'Only Then Shall We Find Courage (1946)']


This seems pretty clean.

Now let's see the length of the quotes and the sources

In [42]:
df['quote_length'] = df['quote'].apply(lambda x: len(str(x)))
df['source_length'] = df['source'].apply(lambda x: len(str(x)))

# Preview
df[['quote_length', 'source_length']].describe()


,quote_length,source_length
count,30000.000000,30000.000000
mean,328.241067,90.450633
std,392.587922,167.595513
min,1.000000,1.000000
25%,83.000000,5.000000
50%,187.000000,40.000000
75%,433.000000,112.250000
max,14814.000000,4356.000000


* The longest quotes is 14814 char long and the shortest quote is 1 char long.
* The longest source is 4356 char long and the shortest source is 1 char long.
Let's see them.

In [82]:
# Longest quotes
df.nlargest(5, 'quote_length')[['quote','author','source' ,'heading_context','quote_length']]

quote  \
9023   Judge: Do you want Mr. Bryan sworn? :Darrow: No. :Bryan: I can make affirmation; I can say "So help me God, I will tell the truth. ": Darrow: No, I take it you will tell the truth, Mr. Bryan. You have given considerable study to the Bible, haven't you, Mr. Bryan? :Bryan: Yes, sir, I have tried to. :Darrow: Then you have made a general study of it? :Bryan: Yes, I have; I have studied the Bible for about 50 years, or sometime more than that, but, of course, I have studied it more as I have become older than when I was but a boy. :Darrow: You claim that everything in the Bible should be literally interpreted? :Bryan: I believe everything in the Bible should be accepted as it is given there: some of the Bible is given illustratively. For instance: "Ye are the salt of the earth. " I would not insist that man was actually salt, or that he had flesh of salt, but it is used in the sense of salt as saving God's people. :Darrow: But when you read that Jonah swallowed the whale--or that the whale swallowed Jonah--excuse me please--how do you literally interpret that? :Bryan: When I read that a "big fish" swallowed Jonah--it does not say whale. That is my recollection of it. A big fish, and I believe it, and I believe in a God who can make a whale and can make a man and make both what He pleases. :Darrow: Now, you say, the big fish swallowed Jonah, and he there remained how long--three days--and then he spewed him upon the land. You believe that the big fish was made to swallow Jonah? :Bryan: I am not prepared to say that; the Bible merely says it was done. :Darrow: You don't know whether it was the ordinary run of fish, or made for that purpose? :Bryan: You may guess; you evolutionists guess. . :Darrow: You are not prepared to say whether that fish was made especially to swallow a man or not? :Bryan: The Bible doesn't say, so I am not prepared to say. :Darrow: But do you believe He made them--that He made such a fish and that it was big enough to swallow Jonah? :Bryan: Yes, sir. Let me add: One miracle is just as easy to believe as another. :Darrow: Just as hard? :Bryan: It is hard to believe for you, but easy for me. A miracle is a thing performed beyond what man can perform. When you get within the realm of miracles; and it is just as easy to believe the miracle of Jonah as any other miracle in the Bible. :Darrow: Perfectly easy to believe that Jonah swallowed the whale? :Bryan: If the Bible said so; the Bible doesn't make as extreme statements as evolutionists do. :Darrow: The Bible says Joshua commanded the sun to stand still for the purpose of lengthening the day, doesn't it, and you believe it. :Bryan: I do. :Darrow: Do you believe at that time the entire sun went around the earth? :Bryan: No, I believe that the earth goes around the sun. :Darrow: Do you believe that the men who wrote it thought that the day could be lengthened or that the sun could be stopped? :Bryan: I don't know what they thought. :Darrow: You don't know? :Bryan: I think they wrote the fact without expressing their own thoughts. :Darrow: Have you an opinion as to whether or not the men who wrote that thought--: Thomas Stewart: (a prosecution lawyer)--I want to object, your honor. It has gone beyond the pale of any issue that could possibly be injected into this lawsuit, except by imagination. I do not think the defendant has a right to conduct the examination any further and I ask your honor to exclude it. :Bryan: It seems to me it would be too exacting to confine the defense to the facts. If they are not allowed to get away from the facts, what have they to deal with? :Judge: Mr. Bryan is willing to be examined. Go ahead. :Darrow: Can you answer my question directly? If the day was lengthened by stopping either the earth or the sun, it must have been the earth? :Bryan: Well, I should say so. :Darrow: Now, Mr. Bryan, have you ever pondered what would have happened to the earth if it had stood still? :Bryan: No. :Darrow: You have not? :Bryan: No; 

Oh, the first one is a court trial so that's why it is very long.

In [47]:
# Longest sources
df.nlargest(5, 'source_length')[['quote' , 'author' , 'source', 'source_length']]

quote  \
419                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              We cannot solve the problems using the same kind of thinking we used when we created them   
417                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Two things are infinite: the universe and the human stupidity.   
8363   中国人民从中国解放区和国民党统治区，获得了明显的比较。难道还不明显吗？两条路线，人民战争的路线和反对人民战争的消极抗日的路线，其结果：一条是胜利的，即使处在中国解放区这种环境恶劣和毫无外援的地位；另一条是失败的，即使处在国民党统治区这种极端有利和取得外国接济的地位。国民党政府把自己的失败归咎于缺乏武器。但是试问：缺乏武器的是国民党的军队呢，还是解放区的军队？中国解放区的军队是中国军队中武器最缺乏的军队，他们只能从敌人手里夺取武器和在最恶劣条件下自己制造武器。国民党中央系军队的武器，不是比起地方系军队来要好得多吗？但是比起战斗力来，中央系却多数劣于地方系。国民党拥有广大的人力资源，但是在它的错误的兵役政策下，人力补充却极端困难。中国解放区处在被敌人分割和战斗频繁的情况之下，因为普遍实施了适合人民需要的民兵和自卫军制度，又防止了对于人力资源的滥用和浪费，人力动员却可以源源不竭。国民党拥有粮食丰富的广大地区，人民每年供给它七千万至一万万市担的粮食，但是大部分被经手人员中饱了，致使国民党的军队经常缺乏粮食，士兵饿得面黄肌瘦。中国解放区的主要部分隔在敌后，遭受敌人烧杀抢"三光"政策的摧残，其中有些是像陕北这样贫瘠的区域，但是却能用自己动手、发展农业生产的方法，很好地解决了粮食问题。国民党区域经济危机极端严重，工业大部分破产了，连布匹这样的日用品也要从美国运来。中国解放区却能用发展工业的方法，自己解决布匹和其他日用品的需要。在国民党区域，工人、农民、店员、公务人员、知识分子以及文化工作者，生活痛苦，达于极点。中国解放区的全体人民都有饭吃，有衣穿，有事做。利用抗战发国难财，官吏即商人，贪污成风，廉耻扫地，这是国民党区域的特色之一。艰苦奋斗，以身作则，工作之外，还要生产，奖励廉洁，禁绝贪污，这是中国解放区的特色之一。国民党区域剥夺人民的一切自由。中国解放区则给予人民以充分的自由。国民党统治者面前摆着这些反常的状况，怪谁呢？怪别人，还是怪他们自己呢？怪外国缺少援助，还是怪国民党政府的独裁统治和腐败无能呢？这难道还不明白吗？   
13790                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Anyone who is not shocked by quantum theory has not understood it.   
9755                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

Extraccting the usefull information from these long sources and the quotes to trim it is very complex. I think we should drop them.

One more operation we could perform in the preprocessing part is that for the sources which have NaN value, we could check if the quotes have :* this pattern in them. Because it represents the continuation of it. SOmetimes it contains the actual source for examples in the first longest quote. Although that quote is not useful in our case we atlest should try to split the quote in that and if it has someting we will put it into the source.

In [49]:
# Shortest quotes
df.nsmallest(5, 'quote_length')[['quote' , 'author' , 'source', 'quote_length']]

,quote,author,source,quote_length
579,…,Wikiquote: Templates,NaN,1
4768,.,Anonymous,"Attributed to Joseph P. Kennedy Sr. in J. H. Cutler, Honey Fitz (1962), p. 291; also attributed to Knute Rockne, and others",1
5387,',Aristotle,"Every realm of nature is marvellous: and as Heraclitus, when the strangers who came to visit him found him warming himself at the furnace in the kitchen and hesitated to go in, is reported to have bidden them not to be afraid to enter, as even in that kitchen divinities were present, so we should venture on the study of every kind of animal without distaste; for each and all will reveal to us something natural and something beautiful. Book I, Part 5",1
5436,',Aristotle,"Virtue then is a settled disposition of the mind as regards the choice of actions and feelings, consisting essentially in the observance of the mean relative to us, this being determined by principle, that is, as the prudent man would determine it. And it is a mean state between two vices, one of excess and one of defect. Book II, 1106b. 28-1107a. 3. Rackham, . On Golden mean (philosophy)#Aristotle.",1
5866,',Cicero,"But of what immense worth is it for the soul to be with itself, to live, as the phrase is, with itself, discharged from the service of lust, ambition, strife, enmities, desires of every kind! If one has some provision laid up, as it were, of study and learning, nothing is more enjoyable than the leisure of old age. XIV, 49 (Latin, Peabody, Falconer) ""'"" translated to English as ""leisure of old age"" (Peabody) or ""leisured old age"" (Falconer), translated to Japanese as "" (Yoshida 1950). Alternate translation (Falconer): But how blessed it is for the soul, after having, as it were, finished its campaigns of lust and ambition, of strife and enmity and of all the passions, to return within itself, and, as the saying is, ""to live apart""! And indeed if it has any provender, so to speak, of study and learning, nothing is more enjoyable than a leisured old age.",1


In [53]:
# Shortest quotes
df.nsmallest(5, 'source_length')[['quote' , 'author' , 'source', 'source_length', 'heading_context']]

,quote,author,source,source_length,heading_context
2739,This is a simple story of a battle; such a tale as may be told by a soldier who is no writer to a reader who is no soldier.,Ambrose Bierce,I,1,What I Saw At Shiloh (1881)
2740,"An army's bravest men are its cowards. The death which they would not meet at the hands of the enemy they will meet at the hands of their officers, with never a flinching.",Ambrose Bierce,V,1,What I Saw At Shiloh (1881)
2741,"Hidden in hollows and behind clumps of rank brambles were large tents, dimly lighted with candles, but looking comfortable. The kind of comfort they supplied was indicated by pairs of men entering and reappearing, bearing litters; by low moans from within and by long rows of dead with covered faces outside. These tents were constantly receiving the wounded, yet were never full; they were continually ejecting the dead, yet were never empty. It was as if the helpless had been carried in and murdered, that they might not hamper those whose business it was to fall to-morrow.",Ambrose Bierce,V,1,What I Saw At Shiloh (1881)
6011,"Let the superior man never fail reverentially to order his own conduct, and let him be respectful to others and observant of propriety: —then all within the four seas, all men are brothers. What has the superior man to do with being distressed because he has no brothers?",Confucius,V,1,Analects
8280,"When you talk with famous scholars, the best thing is to pretend that occasionally you do not quite understand them. If you understand too little, you will be despised; if you understand too much, you will be disliked; if you just fail occasionally to understand them you will suit each other very well.",Lu Xun,2,1,"""The Epigrams of Lusin"""


* The problem with the 1 length quote and source is that in the quote part the quote is in another language so it has only scrapped the full stop. And all the following lists have been scrapped in the source it is solvable.
* Same with the source length 1. The source len 1 is the chapter and the heading contexxt is the book. So with both it becomes informative.

Lets see how many wuotes have smaller len that 5 and how many sources have smaller length than 5

In [54]:
short_quotes_count = (df['quote'].apply(lambda x: len(str(x))) < 5).sum()

# Count sources shorter than 5 characters
short_sources_count = (df['source'].apply(lambda x: len(str(x))) < 5).sum()

print(f"Quotes shorter than 5 chars: {short_quotes_count}")
print(f"Sources shorter than 5 chars: {short_sources_count}")

Quotes shorter than 5 chars: 44
Sources shorter than 5 chars: 7336


In [55]:
short_quotes_df = df[df['quote'].apply(lambda x: len(str(x)) < 5)]
print(short_quotes_df[['quote' , 'author' , 'source', 'heading_context']])

      quote                author  \
579       …  Wikiquote: Templates   
583     . .  Wikiquote: Templates   
4768      .             Anonymous   
5387      '             Aristotle   
5436      '             Aristotle   
5866      '                Cicero   
6019      '             Confucius   
7153    FAQ  Wikiquote: Utilities   
7155   Logo  Wikiquote: Utilities   
13554  ". "         Misquotations   
13703   NaN      Chinese proverbs   
13704   NaN      Chinese proverbs   
13707     ,      Chinese proverbs   
13708   NaN      Chinese proverbs   
13709   NaN      Chinese proverbs   
13710     ，      Chinese proverbs   
13712   NaN      Chinese proverbs   
13713   NaN      Chinese proverbs   
13714   NaN      Chinese proverbs   
13715   NaN      Chinese proverbs   
13716     一      Chinese proverbs   
13717   NaN      Chinese proverbs   
13718   NaN      Chinese proverbs   
13719     ，      Chinese proverbs   
13720   NaN      Chinese proverbs   
13721   NaN      Chinese proverbs   
1

most of them are meaningfull no need to delete all. We should certailny drop which author has wikiquote3 in it and others need to be checked.

now lets see the number of quotes for whom the source len is less than 2 adn the heading context is nan

In [64]:
mask = df['source'].apply(lambda x: len(str(x)) < 10) & df['heading_context'].isna()

count = mask.sum()
print("Number of quotes:", count)

# If you also want to see them:
print(df[mask][['quote', 'source', 'heading_context']])

Number of quotes: 777
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      quote  \
421                                                                                                                                                                                                                                

In [65]:
df[mask]['source'].unique()

array([nan, 'Couplet', 'Scene i', 'Scene iv', 'Scene ii', 'Scene I',
       'Scene III', 'Scene VII', 'Lennox', 'Scene IV', 'Scene II',
       'Scene iii', 'scene i', 'Prospero', 'Scene VI', "'.", 'Ibid.',
       'Scene xiv', 'p. 11', 'p. 12', 'p. 13', 'p. 15', 'p. 17', 'P. 19',
       'p. 28', 'p. 29', 'p. 32', 'p. 50', 'p. 50-51', 'p. 52', 'p. 54',
       'p. 58', 'p. 59', 'Cited in', '17.', 'Chapter 1', 'Chapter 2',
       'Chapter 3', 'Chapter 4', 'Chapter 5', 'Chapter 7', 'Chapter 9',
       'Epilogue', '.', '(2006).', 'Minotaur', 'Cicero', '{{cite',
       'Björk', 'proverb', ', p. xcv', '<', ', p. 70', ', p. 71',
       ', p. 97', ', p. 125', ', p. 115', ', p. 105', ', p. 112',
       ', p. 447', ', p. 588', ', p. 788', ', p. 657', ', p. 698',
       ', p. 463', ', p. 171', ', p. 602', ', p. 264', ', p. 512',
       ', p. 709', ', p. 723', ', p. 752', ', p. 734', ',', 'Daniel'],
      dtype=object)

There are so many we need to preprocess these too. The solution could be to search for source in the quote.
* same techique of splitting from the last encounterd :* and saving it as source.
*  We will do that in preprocessing

In [71]:
quote_len_99 = df['quote_length'].quantile(0.99)
source_len_99 = df['source_length'].quantile(0.99)

print("99th percentile quote length:", quote_len_99)
print("99th percentile source length:", source_len_99)

99th percentile quote length: 1784.0199999999968
99th percentile source length: 751.0


We need to dela with this also.

Also the thing we have to decide is that wgether we have to make node for all four features or just quote feature and all other are properties. In which way we can increase the speed and save memory.

Let's see what percentage of the source starts with translation.

In [93]:
pattern = re.compile(r'^\s*(translation|translations|translated|transliteration|transliterated|english equivalent|meaning|literal translation|pinyin:|english:|Original Latin:|)\b', re.IGNORECASE)

# check which rows match
df['starts_with_translation'] = df['source'].fillna("").apply(lambda x: bool(pattern.match(str(x))))

# calculate percentage
percentage = df['starts_with_translation'].mean() * 100

print(f"{percentage:.2f}% of sources start with 'translation', 'transliteration', or a variation")


2.11% of sources start with 'translation', 'transliteration', or a variation


This is a good number (500). How to utilise this?

In [94]:
df[df['starts_with_translation']][['quote', 'source','author','heading_context']].head(10)

,quote,source,author,heading_context
244,"Einer, der nur Zeitungen liest und, wenn's hochkommt, Bücher zeitgenössischer Autoren, kommt mir vor wie ein hochgradig Kurzsichtiger, der es verschmäht, Augengläser zu tragen. Er ist völlig abhängig von den vorurteilen und Moden seiner Zeit, denn er bekommt nichts anderes zu sehen und zu hören. Und was einer selbständig denkt ohne Anlehnung an das Denken und Erleben anderer, ist auch im besten Falle Ziemlich ärmlich und monoton.","Translation: Somebody who reads only newspapers and at best books of contemporary authors appears to me like an extremely near-sighted person who scorns eyeglasses. He is completely dependent on the prejudices and fashions of his times, since he never gets to see or hear anything else. And what a person thinks on his own, without being stimulated by the thoughts and experiences of other people, is, similarly, even in the best case rather paltry and monotonous. Article in Der Jungkaufmann, April 1952, Einstein Archives 28-972",Albert Einstein,1950s
565,Non-English quote.,"Translation: English Translation Source: Chapter xx, sentence xx or Act xx, Scene xx. Optional clarifications, notes on context, etc.",Wikiquote: Templates,NaN
586,Foreign language quote.,Translation: English translation Author and source,Wikiquote: Templates,NaN
601,"A diabolo, qui est simia dei.","Translation: ""From the devil, who is a monkey god. "" English equivalent: Where god has a church the devil will have his chapel. ""Wherever God erects a house of prayer, The Devil always builds a chapel there: And 'twill be found, upon examination, The latter has the largest congregation. "" ""where there's good there's also Evil"" Daniel Defoe, The True-Born Englishman (1701) Source for proverb:",Latin proverbs,NaN
604,Acquirit qui tuetur.,English equivalent: Sparing is the first gaining.,Latin proverbs,NaN
607,Aegrescit medendo.,English equivalent: The remedy is often worse than the disease.,Latin proverbs,NaN
608,"Aegroto dum anima est, spes est.",English equivalent: As long as there is life there is hope.,Latin proverbs,NaN
609,Aeque pars ligni curvi ac recti valet igni.,English equivalent: Crooked logs make straight fires.,Latin proverbs,NaN
611,"Aliis si licet, tibi non licet.","Translation: If others are allowed to, that does not mean you are. (see also quod licet Iovi, non licet bovi)",Latin proverbs,NaN
612,"An nescis, mi fili, quantilla prudentia mundus regatur? (alternatively: regatur orbis)","Translation: Don't you know, my son, with how little wisdom the world is governed? (1583 – 1654), 1648 letter to son, who was involved in negotiating the Classical and foreign quotations, William Francis Henry King, 1889, p. 40, quote #300 Sometimes attributed to Cardinal Richelieu. Variant form due to John Selden",Latin proverbs,NaN


What we can do is to create another property "translation" of quote nodes. And utilise this in that way. Then we will see what amount of quotes have null value of len less than 5.

Next step in the EDA is to check if the quotes or the source contains links or not.

In [78]:
url_pattern = re.compile(r'https?://\S+')

# Boolean columns for detection
df['quote_has_link'] = df['quote'].fillna("").apply(lambda x: bool(url_pattern.search(x)))
df['source_has_link'] = df['source'].fillna("").apply(lambda x: bool(url_pattern.search(x)))

# Summary counts
print("Quotes with links:", df['quote_has_link'].sum())
print("Sources with links:", df['source_has_link'].sum())

# Percentage
print("Quotes with links (%):", df['quote_has_link'].mean() * 100)
print("Sources with links (%):", df['source_has_link'].mean() * 100)

Quotes with links: 0
Sources with links: 0
Quotes with links (%): 0.0
Sources with links (%): 0.0


They don't have typical links because `I took care of that while extracting the information. But there is a pattern I noticed which is has not been taken care of
* {{ cite book

In [91]:
pattern = r"\{\{\s*cite\s+book"
count = df['source'].fillna("").str.contains(pattern, case=False, regex=True).sum()

print(f"Number of sources containing '{{{{cite book}}}}': {count}")


Number of sources containing '{{cite book}}': 47


In [92]:
matches = df[df['source'].fillna("").str.contains(pattern, case=False, regex=True)]
print(matches[['quote', 'source']])

There are 47 quotes which has {{ cite book in it but the one which ends with it are meaning less. So we will delete only the quotes which ends with {{cite boo and leave others.

Let's see if we have any duplicate quotes

In [100]:
# Find duplicates based on BOTH quote and source
duplicates = df[df.duplicated(subset=['quote', 'source', 'author'], keep=False)]
# pd.set_option('display.max_colwidth', None)
# See how many there are
print(f"Total duplicate rows (quote+source): {duplicates.shape[0]}")

# View some examples
duplicates.sort_values(by='quote').head(20)

Total duplicate rows (quote+source): 88


,quote,source,author,heading_context,quote_length,source_length,starts_with_translation,quote_has_link,source_has_link
23577,"""Well done"" is better than ""well said"".",NaN,English proverbs,NaN,39,3,False,False,False
24081,"""Well done"" is better than ""well said"".",NaN,English proverbs,NaN,39,3,False,False,False
566,(. . quote. . ),NaN,Wikiquote: Templates,NaN,15,3,False,False,False
567,(. . quote. . ),NaN,Wikiquote: Templates,NaN,15,3,False,False,False
568,(. . quote. . ),NaN,Wikiquote: Templates,NaN,15,3,False,False,False
569,(. . quote. . ),NaN,Wikiquote: Templates,NaN,15,3,False,False,False
23389,Accidents will happen in the best families. (19th century),"Citatboken, Bokförlaget Natur och Kultur, Stockholm, 1967, p. 187, ISBN 91-27-01681-1",English proverbs,NaN,58,85,False,False,False
23659,Accidents will happen in the best families. (19th century),"Citatboken, Bokförlaget Natur och Kultur, Stockholm, 1967, p. 187, ISBN 91-27-01681-1",English proverbs,NaN,58,85,False,False,False
22000,"After dinner Mr. Mill read us Shelley's Ode to Liberty & he got quite excited & moved over it rocking backwards & forwards & nearly choking with emotion; he said himself: ""it is almost too much for one. ""","Lord Amberley, journal entry (28 September 1870), quoted in Bertrand and Patricia Russell (eds. ), The Amberley Papers, Volume II (1937), p. 375",unknown,NaN,204,144,False,False,False
25735,"After dinner Mr. Mill read us Shelley's Ode to Liberty & he got quite excited & moved over it rocking backwards & forwards & nearly choking with emotion; he said himself: ""it is almost too much for one. ""","Lord Amberley, journal entry (28 September 1870), quoted in Bertrand and Patricia Russell (eds. ), The Amberley Papers, Volume II (1937), p. 375",unknown,NaN,204,144,False,False,False


There are multiple duplicates in the dataset. We have to remove them also

Now I think the EDA is complete let's preprocess the dataframe 

### Preprocessing

Let's write down the steps in the preprocessing.
* I have to deal with the rows corresponding to the null quote values. Either extract quopte from the sourcce or remove them.(most of them are chinese proverbs)
* delete wikiquote pages
* deal with unit len quote and source
* deal with very large quote and source
* One more operation we could perform in the preprocessing part is that for the sources which have NaN value, we could check if the quotes have :* this pattern in them. Because it represents the continuation of it. Sometimes it contains the actual source for examples in the first longest quote. Although that quote is not useful in our case we atlest should try to split the quote in that and if it has someting we will put it into the source.
* check if the source has only [nan, 'Couplet', 'Scene i', 'Scene iv', 'Scene ii', 'Scene I',
       'Scene III', 'Scene VII', 'Lennox', 'Scene IV', 'Scene II',
       'Scene iii', 'scene i', 'Prospero', 'Scene VI', "'.", 'Ibid.',
       'Scene xiv', 'p. 11', 'p. 12', 'p. 13', 'p. 15', 'p. 17', 'P. 19',
       'p. 28', 'p. 29', 'p. 32', 'p. 50', 'p. 50-51', 'p. 52', 'p. 54',
       'p. 58', 'p. 59', 'Cited in', '17.', 'Chapter 1', 'Chapter 2',
       'Chapter 3', 'Chapter 4', 'Chapter 5', 'Chapter 7', 'Chapter 9',
       'Epilogue', '.', '(2006).', 'Minotaur', 'Cicero', '{{cite',
       'Björk', 'proverb', ', p. xcv', '<', ', p. 70', ', p. 71',
       ', p. 97', ', p. 125', ', p. 115', ', p. 105', ', p. 112',
       ', p. 447', ', p. 588', ', p. 788', ', p. 657', ', p. 698',
       ', p. 463', ', p. 171', ', p. 602', ', p. 264', ', p. 512',
       ', p. 709', ', p. 723', ', p. 752', ', p. 734', ',', 'Daniel'],
      dtype=object) and no heading context or author. If these two are not present the source itself is not meaning full. so we have to deal with it. Or see if the quotes contains source :* after this pattern. if yes then I will extract that form the quote and put in the source/
* Also the thing we have to decide is that wgether we have to make node for all four features or just quote feature and all other are properties. In which way we can increase the speed and save memory.
* 2.11% of sources start with 'translation', 'transliteration', or a variation. What we can do is to create another property "translation" of quote nodes. And utilise this in that way. Then we will see what amount of quotes have null value of len less than 5.
* There are 47 quotes which has {{ cite book in it but the one which ends with it are meaning less. So we will delete only the quotes which ends with {{cite boo and leave others.
* There are 88 duplicate rows need to delete them.

Remove htmls. wikiquotes authers, some patterns from the quote and source text.

So we have two options for chinese proverbs

to drop the columns for which the quote is NaN value.
to switch the source and the quote value where the quote is NaN. Because the source contains the translation and in a way it is also a quote. we will decide in the preprocessing part

We should do something for  Albert einstien and misattributed to ALbert Einstien and disputed with albert Einstin are three different authors in the author list. Which is a entity fragmentation. Wer should think like we should put the author as a node in the databse or the property of the quote in the databse. If the author is a node then these three variations should be replaced with one author node and the misattributed and disputed should be property of code for better memory utilisation

One more operation we could perform in the preprocessing part is that for the sources which have NaN value, we could check if the quotes have :* this pattern in them. Because it represents the continuation of it. SOmetimes it contains the actual source for examples in the first longest quote. Although that quote is not useful in our case we atlest should try to split the quote in that and if it has someting we will put it into the source.

Drop wikiquote templates


We will check the single length quote rows and max len in the preprocessing

First of all let's drop the wikiquote template and duplicates

In [26]:
df = df[df["author"].ne("Wikiquote: Templates")].reset_index(drop=True)
df = df[df["author"].ne("Wikiquote: Requests")].reset_index(drop=True)
df = df[df["author"].ne("Wikiquote: Utilities")].reset_index(drop=True)
df = df.drop_duplicates(subset=["quote", "source", "heading_context"]).reset_index(drop=True)


ALso let's remove every entry related to wikkiquote as author

In [38]:
# Find rows where author starts with "Wikiquote:"
wikiquote_authors = df_new[df_new["author"].astype(str).str.startswith("Wikiquote:")]
wikiquote_authors


,quote,source,author,heading_context
3926,Famous speeches of historical value can be added to the speeches portal at Wikisource.,NaN,Wikiquote: What Wikiquote is not,NaN
3927,"Whole public domain books can be added to Wikisource or Wikibooks, see Wikisource Project: Annotations.",NaN,Wikiquote: What Wikiquote is not,NaN
3928,Generic deposits of published information may be appropriate at Wikisource.,NaN,Wikiquote: What Wikiquote is not,NaN
3929,"Book PDFs/DJVUs, pictures, sound files, and other forms of media that are in the public domain or under a libre license are accepted at Wikimedia Commons.",NaN,Wikiquote: What Wikiquote is not,NaN
3930,Wikipedia: What Wikipedia is not,NaN,Wikiquote: What Wikiquote is not,NaN
...,...,...,...,...
25713,Themebot - whether we should have an automatic theme generator. (abandoned concept),NaN,Wikiquote: Issues,NaN
25714,Logo - what the Wikiquote logo should be. (has been chosen) Wikiquote|,NaN,Wikiquote: Issues,NaN
26628,"""sourced quote""",NaN,Wikiquote: Sourced and Unsourced sections,NaN
26629,"""moved to Misattributed after finding original quote""",NaN,Wikiquote: Sourced and Unsourced sections,NaN


In [92]:
df = df[~df["author"].astype(str).str.startswith("Wikiquote:")].reset_index(drop=True)
df = df[~df["author"].astype(str).str.startswith("List of categories")].reset_index(drop=True)
# List of categories

In [93]:
df["quote"] = df["quote"].replace("NaN", np.nan)

# Regex: only spaces or punctuation (no letters/numbers)
only_special_pattern = r'^[^\w\u4e00-\u9fff]+$'  # excludes alphanumerics + Chinese chars

mask = (
    df["quote"].isna() | 
    df["quote"].astype(str).str.fullmatch(only_special_pattern)
)

# Filter
special_or_nan_quotes = df[mask]

print(f"Found {len(special_or_nan_quotes)} rows")
display(special_or_nan_quotes)

Found 32 rows


,quote,source,author,heading_context
4656,.,"Attributed to Joseph P. Kennedy Sr. in J. H. Cutler, Honey Fitz (1962), p. 291; also attributed to Knute Rockne, and others",Anonymous,NaN
5275,',"Every realm of nature is marvellous: and as Heraclitus, when the strangers who came to visit him found him warming himself at the furnace in the kitchen and hesitated to go in, is reported to have bidden them not to be afraid to enter, as even in that kitchen divinities were present, so we should venture on the study of every kind of animal without distaste; for each and all will reveal to us something natural and something beautiful. Book I, Part 5",Aristotle,Parts of Animals
5324,',"Virtue then is a settled disposition of the mind as regards the choice of actions and feelings, consisting essentially in the observance of the mean relative to us, this being determined by principle, that is, as the prudent man would determine it. And it is a mean state between two vices, one of excess and one of defect. Book II, 1106b. 28-1107a. 3. Rackham, . On Golden mean (philosophy)#Aristotle.",Aristotle,Book II
5754,',"But of what immense worth is it for the soul to be with itself, to live, as the phrase is, with itself, discharged from the service of lust, ambition, strife, enmities, desires of every kind! If one has some provision laid up, as it were, of study and learning, nothing is more enjoyable than the leisure of old age. XIV, 49 (Latin, Peabody, Falconer) ""'"" translated to English as ""leisure of old age"" (Peabody) or ""leisured old age"" (Falconer), translated to Japanese as "" (Yoshida 1950). Alternate translation (Falconer): But how blessed it is for the soul, after having, as it were, finished its campaigns of lust and ambition, of strife and enmity and of all the passions, to return within itself, and, as the saying is, ""to live apart""! And indeed if it has any provender, so to speak, of study and learning, nothing is more enjoyable than a leisured old age.",Cicero,Cato Maior de Senectute – On Old Age (44 BC)
5907,',"The Master said, ""Hard is it to deal with him, who will stuff himself with food the whole day, without applying his mind to anything good! Are there not gamesters and chess players? To be one of these would still be better than doing nothing at all. "" Book XVII, Chapter XXII ""'"" means ""to gamble"" in Modern Chinese, and ""to play Go; to play chess"" in Classical Chinese (博弈, 博奕). It is translated as ""gamesters and chess players"" (Legge 1861), ""play at chequers"" (Lyall 1909), and ""checker or chess players"" (Soothill 1910).",Confucius,Analects
13404,""". ""","This is often erroneously assumed to be the quote of Ben Parker dating back to the original Spider-Man origin story as depicted in 1962's Amazing Fantasy #15. This statement appears as a caption of narration in the last panel of the story and was not a spoken line by any character in the story. In most retellings of Spider-Man's origin, including the 2002 film, the quote has been retconned (the alteration of previously established facts in the continuity of a fictional work) to depict Uncle Ben's final lecture to Peter Parker before Ben's tragic death and as the words that continue to drive Peter as Spider-Man. Also, the correct Amazing Fantasy quote is, ""With great power there must also come great responsibility. """,Misquotations,NaN
13549,， ， ， ； 。,"Transliteration (pinyin): Bù wén bù ruò wén zhī, wén zhī bù ruò jiàn zhī, jiàn zhī bù ruò zhīzhī, zhīzhī bù ruò xíng zhī; xué zhìyú xíng zhī ér zhǐ yǐ. Traditional: 不聞不若聞之，聞之不若見之，見之不若知之，知之不若行之；學至於行之而止矣 Simplified: 不闻不若闻之，闻之不若见之，见之不若知之，知之不若行之；学至于行之而止矣 From Xun Zi (荀子 8. 儒效 23）.",Chinese proverbs,B
13553,NaN,Transliteration (pinyin): Chángjiāng hòulàng tuī qiánlàng. Traditional: 長江後浪推前浪 Simplified: 长江后浪推前浪 Meaning: The energy of the old generation inspires the new.,Chinese proverbs,Ch
13554,NaN,Transliteration (pinyin): Dú wàn juǎn shū bùrú xíng wànlǐ lù. Traditional: 讀萬卷書不如行萬裡路 Simplified: 读万卷书不如行万里路 Reading

In [94]:
# Missing values
print(df.isna().sum())

quote                16
source             6846
author                0
heading_context    9150
dtype: int64


We can see that the source has multiple translation, pinyin, and other headings and values. We can convert this to dictionary and use one of the value as the quote if the value of the quote is meaning less.

In [95]:

def parse_kv(text, keys=None):
    if not isinstance(text, str) or not text.strip():
        return {}
    if keys is None:
        keys = [
            "Transliteration (pinyin)", "Transliteration", "Pinyin",
            "Traditional", "Simplified", "Meaning", "Literal",
            "Translation", "Variant", "Notes", "Source","who"
        ]
    key_alt = "|".join(map(re.escape, keys))
    pat = re.compile(rf"(?i)\b({key_alt})\b\s*:", re.DOTALL)
    matches = list(pat.finditer(text))
    if not matches:
        return {}
    out = {}
    for i, m in enumerate(matches):
        key_raw = m.group(1)
        start = m.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(text)
        val = text[start:end].strip()
        val = re.sub(r"\s+", " ", val).strip(" \t\n\r\"'，。；;:,-")
        if key_raw in out and val:
            out[key_raw] = f"{out[key_raw]} | {val}"
        elif val:
            out[key_raw] = val
    return out



In [96]:
df_new = df.copy()

for idx in df_new[mask].index:
    kv = parse_kv(df_new.at[idx, "source"])
    if kv:
        # Take first key's value
        first_val = next(iter(kv.values()))
        df_new.at[idx, "quote"] = first_val

        # Remove that key/value from the source text
        first_key = next(iter(kv.keys()))
        pattern = re.compile(rf"{re.escape(first_key)}\s*:\s*{re.escape(first_val)}", re.IGNORECASE)
        df_new.at[idx, "source"] = pattern.sub("", df_new.at[idx, "source"], count=1).strip()

df_new[mask]

,quote,source,author,heading_context
4656,.,"Attributed to Joseph P. Kennedy Sr. in J. H. Cutler, Honey Fitz (1962), p. 291; also attributed to Knute Rockne, and others",Anonymous,NaN
5275,',"Every realm of nature is marvellous: and as Heraclitus, when the strangers who came to visit him found him warming himself at the furnace in the kitchen and hesitated to go in, is reported to have bidden them not to be afraid to enter, as even in that kitchen divinities were present, so we should venture on the study of every kind of animal without distaste; for each and all will reveal to us something natural and something beautiful. Book I, Part 5",Aristotle,Parts of Animals
5324,',"Virtue then is a settled disposition of the mind as regards the choice of actions and feelings, consisting essentially in the observance of the mean relative to us, this being determined by principle, that is, as the prudent man would determine it. And it is a mean state between two vices, one of excess and one of defect. Book II, 1106b. 28-1107a. 3. Rackham, . On Golden mean (philosophy)#Aristotle.",Aristotle,Book II
5754,',"But of what immense worth is it for the soul to be with itself, to live, as the phrase is, with itself, discharged from the service of lust, ambition, strife, enmities, desires of every kind! If one has some provision laid up, as it were, of study and learning, nothing is more enjoyable than the leisure of old age. XIV, 49 (Latin, Peabody, Falconer) ""'"" translated to English as ""leisure of old age"" (Peabody) or ""leisured old age"" (Falconer), translated to Japanese as "" (Yoshida 1950). Alternate translation (Falconer): But how blessed it is for the soul, after having, as it were, finished its campaigns of lust and ambition, of strife and enmity and of all the passions, to return within itself, and, as the saying is, ""to live apart""! And indeed if it has any provender, so to speak, of study and learning, nothing is more enjoyable than a leisured old age.",Cicero,Cato Maior de Senectute – On Old Age (44 BC)
5907,',"The Master said, ""Hard is it to deal with him, who will stuff himself with food the whole day, without applying his mind to anything good! Are there not gamesters and chess players? To be one of these would still be better than doing nothing at all. "" Book XVII, Chapter XXII ""'"" means ""to gamble"" in Modern Chinese, and ""to play Go; to play chess"" in Classical Chinese (博弈, 博奕). It is translated as ""gamesters and chess players"" (Legge 1861), ""play at chequers"" (Lyall 1909), and ""checker or chess players"" (Soothill 1910).",Confucius,Analects
13404,""". ""","This is often erroneously assumed to be the quote of Ben Parker dating back to the original Spider-Man origin story as depicted in 1962's Amazing Fantasy #15. This statement appears as a caption of narration in the last panel of the story and was not a spoken line by any character in the story. In most retellings of Spider-Man's origin, including the 2002 film, the quote has been retconned (the alteration of previously established facts in the continuity of a fictional work) to depict Uncle Ben's final lecture to Peter Parker before Ben's tragic death and as the words that continue to drive Peter as Spider-Man. Also, the correct Amazing Fantasy quote is, ""With great power there must also come great responsibility. """,Misquotations,NaN
13549,不聞不若聞之，聞之不若見之，見之不若知之，知之不若行之；學至於行之而止矣,"Transliteration (pinyin): Bù wén bù ruò wén zhī, wén zhī bù ruò jiàn zhī, jiàn zhī bù ruò zhīzhī, zhīzhī bù ruò xíng zhī; xué zhìyú xíng zhī ér zhǐ yǐ. Simplified: 不闻不若闻之，闻之不若见之，见之不若知之，知之不若行之；学至于行之而止矣 From Xun Zi (荀子 8. 儒效 23）.",Chinese proverbs,B
13553,長江後浪推前浪,Transliteration (pinyin): Chángjiāng hòulàng tuī qiánlàng. Simplified: 长江后浪推前浪 Meaning: The energy of the old generation inspires the new.,Chinese proverbs,Ch
13554,讀萬卷書不如行萬裡路,Transliteration (pinyin): Dú wàn juǎn shū bùrú xíng wànlǐ lù. Simplified: 读万卷书不如行万里路 Reading ten thousand books is not as useful as traveling ten tho

In [97]:
print(df_new.isna().sum())

quote                 0
source             6846
author                0
heading_context    9150
dtype: int64


deleting the only punctuation quotes

In [98]:
only_punct_pattern = r'^[^\w\u4e00-\u9fff]+$'

# Mask for "only punctuation"
mask_only_punct = df_new["quote"].astype(str).str.fullmatch(only_punct_pattern)

# Preview them before removal
punct_only_rows = df_new[mask_only_punct]
punct_only_rows

# Remove them
df_new = df_new[~mask_only_punct].reset_index(drop=True)

In [99]:
short_quotes_count_new = (df_new['quote'].apply(lambda x: len(str(x))) < 5).sum()

print(f"Quotes shorter than 10 chars: {short_quotes_count_new}")

short_quotes_df_new = df_new[df_new['quote'].apply(lambda x: len(str(x)) < 5)]
short_quotes_df_new[['quote' , 'author' , 'source', 'heading_context']]

Quotes shorter than 10 chars: 10


,quote,author,source,heading_context
13552,父債子還,Chinese proverbs,"Transliteration (pinyin): Fù zhài zǐ huán. Simplified: 父债子还 Father's debt, son to give back. Meaning: The new generation can fix the mistakes made by previous ones. ""New generation can put right the mistakes of the old. "" ""To do the opposite of something is also a form of imitation. "" Georg Lichtenberg, The Waste Books, R. J. Hollingdale trans. (2000), D96.",F
13557,良藥苦口,Chinese proverbs,"Transliteration (pinyin): Liángyào kǔkǒu Simplified: 良药苦口 Good medicine tastes bitter. Meaning: What may be good for us later may be hard for us now. English equivalent: Bitter pills may have blessed effects. ""Present afflictions may tend to our future good. """,L
13560,一,Chinese proverbs,"Transliteration (pinyin): Ròu bāozi dǎ gǒu 一 qù bù huítóu. Traditional: 肉包子打狗一去不回頭 Simplified: 肉包子打狗一去不回头 To hit a dog with a meat-bun, so it leaves never turning around. Meaning: Punishment gives less incentive than a reward.",R
13588,光阴似箭,Chinese proverbs,English equivalent: Time flies like an arrow.,Z
14001,No.,Last words,"Who: Alexander Graham Bell, a Scottish-born scientist, inventor, engineer and innovator who is credited with patenting the first practical telephone. Note: While Alexander Graham Bell was dying, his deaf wife whispered to him, ""Don't leave me. "" Bell responded by signing the word, ""No. """,NaN
14563,V-1.,Last words,"Who: Klaas Meurs Note: Klaas Meurs was the First Officer of KLM Flight 4805, which crashed on take-off on March 27th 1977, killing 583 people when it collided with a Pan Am Boeing 747, killing all 248 on Flight 4805 (including Meurs) and 335 on the Pan Am Aircraft. His last words of ""V-1"" where the Aviation terminology used when an Aircraft is going too fast to cancel the take-off and must take-off. Eight seconds later, both planes collided.",NaN
14727,Yeah,Last words,NaN,NaN
14740,No.,Last words,"Who: Alfred Rosenberg, Nazi ideologist and minister Note: When asked if he had any last words before being executed by hanging.",NaN
14877,Yes.,Last words,Who: Alice B. Toklas Note: Her response when asked if she wanted to die.,NaN
14938,No.,Last words,"Who: H. K. ""Hank"" Williams, American country singer Note: In response to whether he wanted something to eat.",NaN


But first let's delete the rows having the author unknown, source NaN and heading_context = NaN

In [100]:
mask_del = (
    df_new["author"].astype(str).str.strip().str.lower().eq("unknown") &
    (
        df_new["source"].isna() |
        df_new["source"].astype(str).str.fullmatch(only_punct_pattern)
    ) &
    df_new["heading_context"].isna()
)

# Preview before deletion
display(df_new[mask_del])

# Delete them
df_new = df_new[~mask_del].reset_index(drop=True)

,quote,source,author,heading_context
542,"His work revolved around three rules which apply to all science, our problems, and times:",NaN,unknown,NaN
543,": 1. Out of clutter, find simplicity;",NaN,unknown,NaN
544,: 2. From discord make harmony; and finally,NaN,unknown,NaN
2104,Russell is the most gifted Englishman alive.,NaN,unknown,NaN
2105,"Russell doesn't understand the importance of the past, or of tradition, and—he won't qualify.",NaN,unknown,NaN
2432,"""I wish I'd said that""",NaN,unknown,NaN
4447,"Plain language sounds purely objective. On the one hand, it has not the accent of mere vituperation, it is thoroughly dignified; and on the other, it is not the language of a person who is mainly concerned with wangling somebody into believing something. When Mr. Jefferson wrote that one of his associates in Washington's cabinet was ""a fool and a blabber, "" his words, taken in their context, make exactly the same impression of calm, disinterested and objective appraisal as if he had remarked that the man had black hair and brown eyes. Or again, while we are about it, let us examine the most extreme example of this sort of thing that I have so far found in English literature, which is Kent's opinion of Oswald, in King Lear: :: Kent. Fellow, I know thee. :: Osw. What dost thou know me for? :: Kent. A knave; a rascal; an eater of broken meats; a base, proud, shallow, beggarly, three-suited, hundred-pound, filthy, worsted-stocking knave; a lily-livered, action-taking whoreson, glass-gazing, & super-servicable, finical rogue; onetrunk-inheriting slave; one that wouldst be a bawd, in way of good service, and art nothing but the composition of a knave, beggar, coward, pandar, and the son and heir of a mongrel bitch. : Now, considering Kent's character and conduct, as shown throughout the play, I doubt very much that those lines should be taken as merely so much indecent blackguarding. . . an actor who ranted through them in the tone and accent of sheer violent diatribe would ruin his part. Frank Warrin cited those lines the other day, when he was telling me how much he would enjoy a revival of Lear, with our gifted friend Bill Parke cast for the part of Kent. He said, ""Can't you hear Bill's voice growing quieter and quieter, colder and colder, deadlier and deadlier, all the way through that passage? "" Angry as Kent is, and plain as his language is, his tone and manner must carry a strong suggestion of objectivity in order to keep fully up to the dramatist's conception of his role. Kent is not abusing Oswald; he is merely, as we say, ""telling him. "": * Albert Jay Nock, in ""Free Speech and Plain Language"" The Atlantic Monthly (January 1936)",NaN,unknown,NaN
4448,"Lear is a play [that] contains a great deal of veiled social criticism — but it is all uttered either by the Fool, by Edgar when he is pretending to be mad, or by Lear during his bouts of madness. In his sane moments Lear hardly ever makes an intelligent remark. :* George Orwell, in Lear",NaN,unknown,NaN
4638,"Caesar overtook his advanced guard at the river Rubicon, which formed the frontier between Gaul and Italy. Well aware how critical a decision confronted him, he turned to his staff, remarking: :: ""We may still draw back but, once across that little bridge, we shall have to fight it out"": As he stood, in two minds, an apparition of superhuman size and beauty was seen sitting on the river bank playing a reed pipe. A party of shepherds gathered around to listen and, when some of Caesar's men broke ranks to do the same, the apparition snatched a trumpet from one of them, ran down to the river, blew a thunderous blast, and crossed over. Caesar exclaimed: :: ""Let us accept this as a sign from the Gods, and follow where they beckon, in vengeance on our double-dealing enemies. The die is cast. "": He led his army to the farther bank, where he welcomed the tribunes of the people who had fled to him from Rome. Then he tearfully addressed the troops and, ripping open his

In [101]:
# Mask: source length < 5 AND heading_context is NaN
mask_src_short_no_context = (
    df_new["source"].astype(str).str.len() < 5
) & (
    df_new["heading_context"].isna()
)

# Preview the rows
short_source_no_context = df_new[mask_src_short_no_context]
display(short_source_no_context)


,quote,source,author,heading_context
421,Stay away from negative people. They have a problem for every solution.,NaN,Disputed with Albert Einstein,NaN
1539,"Republicans approve of the American farmer, but they are willing to help him go broke. They stand four-square for the American home--but not for housing. They are strong for labor--but they are stronger for restricting labor's rights. They favor minimum wage--the smaller the minimum wage the better. They endorse educational opportunity for all--but they won't spend money for teachers or for schools. They think modern medical care and hospitals are fine--for people who can afford them. . .They think the American standard of living is a fine thing--so long as it doesn't spread to all the people. And they admire the Government of the United States so much that they would like to buy it. − Harry S. Truman, October 13, 1948, St. Paul, Minnesota, Radio Broadcast. https: //www. trumanlibrary. gov/library/public-papers/236/address-st-paul-municipal-auditoriumhttp: //www. presidency. ucsb. edu/ws/index. php? pid=13046https: //www. goodreads. com/quotes/97983-republicans-approve-of-the-american-farmer-but-they-are-willinghttps: //www. snopes. com/politics/quotes/trumangop. asphttps: //spydersden. wordpress. com/2014/11/22/ten-quotes-about-republicans-from-harry-truman/https: //www. nytimes. com/2017/11/24/opinion/republican-taxes-healthcare. html#permid=24954077",NaN,Harry S. Truman,NaN
1540,"If you want a friend in Washington, get a dog.",NaN,Harry S. Truman,NaN
2688,"For there be divers sorts of death—some wherein the body remaineth; and in some it vanisheth quite away with the spirit. This commonly occurreth only in solitude (such is God's will) and, none seeing the end, we say the man is lost, or gone on a long journey—which indeed he hath; but sometimes it hath happened in sight of many, as abundant testimony showeth. In one kind of death the spirit also dieth, and this it hath been known to do while yet the body was in vigor for many years. Sometimes, as is veritably attested, it dieth with the body, but after a season is raised up again in that place where the body did decay.",NaN,Ambrose Bierce,NaN
2689,"On every side of me stretched a bleak and desolate expanse of plain, covered with a tall overgrowth of sere grass, which rustled and whistled in the ​autumn wind with heaven knows what mysterious and disquieting suggestion. Protruded at long intervals above it, stood strangely shaped and somber-colored rocks, which seemed to have an understanding with one another and to exchange looks of uncomfortable significance, as if they had reared their heads to watch the issue of some foreseen event. A few blasted trees here and there appeared as leaders in this malevolent conspiracy of silent expectation.",NaN,Ambrose Bierce,NaN
...,...,...,...,...
27739,"In the end, more than freedom, they wanted security. They wanted a comfortable life, and they lost it all – security, comfort, and freedom. When the Athenians finally wanted not to give to society but for society to give to them, when the freedom they wished for most was freedom from responsibility, then Athens ceased to be free and was never free again. :* This quotation appeared in an article by Margaret Thatcher, ""The Moral Foundations of Society"" (Imprimis, March 1995), which was an edited version of a lecture Thatcher had given at Hillsdale College in November 1994. Here is the actual passage from Thatcher's article: :: [M]ore than they wanted freedom, the Athenians wanted security. Yet they lost everything—security, comfort, and freedom. This was because they wanted not to give to society, but for society to give to them. The freedom they were seeking was freedom from responsibility. It is no wonder, then, that they ceased to be free. In the modern world, we should recall the Athenians' dire fate whenever we confront demands for increased state paternalism. :: The italicized passage above originated with Thatcher. In charact

From the quote extracting the source if it is missing.

In [102]:
# import re
# import numpy as np
# import pandas as pd

# # consider "only punctuation/whitespace" as empty
ONLY_PUNCT_RE = r'^[^\w\u4e00-\u9fff]+$'

# split on variations like ":*", ": *", "：*", multiple asterisks, extra spaces
CONT_SPLIT_RE = re.compile(r'\s*[:：]\s*\*+\s*')

def clean_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = re.sub(r"\s+", " ", s).strip()
    # trim common trailing punctuation clutter
    return s.strip(" \t\n\r\"'，。；;:,-")

def pick_candidate(chunks):
    """
    Choose the best 'source-like' chunk:
    - prefer the longest non-empty, non-punctuation chunk
    - length >= 5 characters (tweakable)
    """
    cleaned = [clean_text(c) for c in chunks if isinstance(c, str)]
    # drop empty / punctuation-only
    cleaned = [c for c in cleaned if c and not re.fullmatch(ONLY_PUNCT_RE, c)]
    if not cleaned:
        return ""
    # choose the longest
    return max(cleaned, key=len)

# 1) Build a mask: source is NaN and quote contains a ':*' continuation
mask_continuation = df_new["source"].isna() & df_new["quote"].astype(str).str.contains(CONT_SPLIT_RE)

# 2) Preview what would change (no mutations yet)
preview = []
for idx in df_new[mask_continuation].index:
    q = df_new.at[idx, "quote"]
    parts = CONT_SPLIT_RE.split(q)
    if len(parts) < 2:
        continue
    # everything after the first chunk are candidates for source
    candidate = pick_candidate(parts[1:])
    if candidate:
        preview.append({
            "index": idx,
            "old_quote": q,
            "new_quote": clean_text(parts[0]),
            "new_source_candidate": candidate
        })

preview_df = pd.DataFrame(preview)
display(preview_df.head(20))  # <- review here
print(f"Rows with candidate source found: {len(preview_df)}")


,index,old_quote,new_quote,new_source_candidate
0,1412,". . WHEREAS, IN THE PAST, THE POINT OF DISAGREEMENT HAS BEEN BETWEEN DISSONANCE AND CONSONANCE, IT WILL BE, IN THE IMMEDIATE FUTURE, BETWEEN NOISE AND SO-CALLED MUSICAL SOUNDS. : THE PRESENT METHODS OF WRITING MUSIC, PRINCIPALLY THOSE WHICH EMPLOY HARMONY AND ITS REFERENCE TO PARTICULAR STEPS IN THE FIELD OF SOUND, WILL BE INADEQUATE FOR THE COMPOSER, WHO WILL BE FACED WITH THE ENTIRE FIELD OF SOUND. :** Quote of John Cage, in: 'The Future of Music: Credo' (1937); SILENCE; lectures and writings by Cage, John', Publisher Middletown, Conn. Wesleyan University Press, June 1961, CREDO/3",". . WHEREAS, IN THE PAST, THE POINT OF DISAGREEMENT HAS BEEN BETWEEN DISSONANCE AND CONSONANCE, IT WILL BE, IN THE IMMEDIATE FUTURE, BETWEEN NOISE AND SO-CALLED MUSICAL SOUNDS. : THE PRESENT METHODS OF WRITING MUSIC, PRINCIPALLY THOSE WHICH EMPLOY HARMONY AND ITS REFERENCE TO PARTICULAR STEPS IN THE FIELD OF SOUND, WILL BE INADEQUATE FOR THE COMPOSER, WHO WILL BE FACED WITH THE ENTIRE FIELD OF SOUND.","Quote of John Cage, in: 'The Future of Music: Credo' (1937); SILENCE; lectures and writings by Cage, John', Publisher Middletown, Conn. Wesleyan University Press, June 1961, CREDO/3"
1,1413,"The composer (organizer of sound) will be faced not only with the entire field of sound but also with the entire field of time. The 'frame' or fraction of a second, following established film technique, will probably be the basic unit in the measurement of time. No rhythm will be beyond the composer's reach. : NEW METHODS WILL BE DISCOVERED, BEARING A DEFINITE RELATION TO SCHOENBERG'S TWELVE-TONE SYSTEM: * In: 'The Future of Music: Credo' (1937); in: 'Silence: lectures and writings by Cage, John', Publisher Middletown, Conn. Wesleyan University Press, June 1961, 4/SILENCE","The composer (organizer of sound) will be faced not only with the entire field of sound but also with the entire field of time. The 'frame' or fraction of a second, following established film technique, will probably be the basic unit in the measurement of time. No rhythm will be beyond the composer's reach. : NEW METHODS WILL BE DISCOVERED, BEARING A DEFINITE RELATION TO SCHOENBERG'S TWELVE-TONE SYSTEM","In: 'The Future of Music: Credo' (1937); in: 'Silence: lectures and writings by Cage, John', Publisher Middletown, Conn. Wesleyan University Press, June 1961, 4/SILENCE"
2,3812,"Able was I ere I saw Elba. :* Credited to ""J. T. R. "" of Baltimore, 1848The Golden Rule and Gazette Of The Union, Saturday July 8 1848, page 30, article titled ""Ingenious arrangement of words""; https: //www. google. co. uk/books/edition/The_Golden_Rule_and_Odd_fellows_Family_C/hEg2AQAAMAAJ? hl=en&gbpv=1&dq=able%20was%20i%20ere%20i%20saw%20elba: * Of such attributions to Napoleon, there is little credence, as stated by William Irvine in Madam I'm Adam and Other Palindromes (1987): ""The well-known ABLE WAS I, ERE I SAW ELBA, for example, is conveniently attributed to Napoleon, whose knowledge of English wordplay was certainly questionable, at best. "" There is no mention of such a palindrome in O'Meara's own work, Napoleon in Exile: or, A Voice from St. Helena (1822).",Able was I ere I saw Elba.,"Of such attributions to Napoleon, there is little credence, as stated by William Irvine in Madam I'm Adam and Other Palindromes (1987): ""The well-known ABLE WAS I, ERE I SAW ELBA, for example, is conveniently attributed to Napoleon, whose knowledge of English wordplay was certainly questionable, at best. "" There is no mention of such a palindrome in O'Meara's own work, Napoleon in Exile: or, A Voice from St. Helena (1822)."
3,4567,"Through all the years that I have been in business I have never yet found our business bad as a result of any outside force. It has always been due to some defect in our own company, and whenever we located and repaired that defect our business became good again - regardless of what anyone else might be doing. And it will always be foun

Rows with candidate source found: 46


In [103]:
for row in preview:
    idx = row["index"]
    df_new.at[idx, "quote"]  = row["new_quote"]
    df_new.at[idx, "source"] = row["new_source_candidate"]

# (optional) tidy any quotes that became empty/punct-only after trimming
mask_bad_quote = df_new["quote"].astype(str).str.fullmatch(ONLY_PUNCT_RE) | df_new["quote"].isna()
# decide whether to drop or leave them; example drops them:
df_new = df_new[~mask_bad_quote].reset_index(drop=True)

In [104]:
mask_bad_quote

0        False
1        False
2        False
3        False
4        False
         ...  
29670    False
29671    False
29672    False
29673    False
29674    False
Name: quote, Length: 29675, dtype: bool

Now let's see the rows which have sources length less than 5 but other than NaN

In [105]:
# rows with non-NaN source and length < 5 (after stripping whitespace)
mask_src_short = df_new["source"].notna() & (df_new["source"].astype(str).str.strip().str.len() < 5)

short_sources = df_new[mask_src_short]
print(f"Rows found: {len(short_sources)}")
display(short_sources)

Rows found: 248


,quote,source,author,heading_context
355,"School failed me, and I failed the school. It bored me. The teachers behaved like Feldwebel (sergeants). I wanted to learn what I wanted to know, but they wanted me to learn for the exam. What I hated most was the competitive system there, and especially sports. Because of this, I wasn't worth anything, and several times they suggested I leave. This was a Catholic School in Munich. I felt that my thirst for knowledge was being strangled by my teachers; grades were their only measurement. How can a teacher understand youth with such a system? . . from the age of twelve I began to suspect authority and distrust teachers. I learned mostly at home, first from my uncle and then from a student who came to eat with us once a week. He would give me books on physics and astronomy. The more I read, the more puzzled I was by the order of the universe and the disorder of the human mind, by the scientists who didn't agree on the how, the when, or the why of creation. Then one day this student brought me Kant's Critique of Pure Reason. Reading Kant, I began to suspect everything I was taught. I no longer believed in the known God of the Bible, but rather in the mysterious God expressed in nature.",p. 8,Albert Einstein,Einstein and the Poet (1983)
1636,"The conception of the necessary unit of all that is resolves itself into the poverty of the imagination, and a freer logic emancipates us from the straitwaistcoated benevolent institution which idealism palms off as the totality of being.",p. 9,Bertrand Russell,Our Knowledge of the External World (1914)
1637,"The true function of logic. . as applied to matters of experience. . is analytic rather than constructive; taken a priori, it shows the possibility of hitherto unsuspected alternatives more often than the impossibility of alternatives which seemed prima facie possible. Thus, while it liberates imagination as to what the world may be, it refuses to legislate as to what the world is.",p. 8,Bertrand Russell,Our Knowledge of the External World (1914)
1985,"When I was a child the atmosphere in the house was one of puritan piety and austerity. There were family prayers at eight o'clock every morning. Although there were eight servants, food was always of Spartan simplicity, and even what there was, if it was at all nice, was considered too good for children. For instance, if there was apple tart and rice pudding, I was only allowed the rice pudding. Cold baths all the year round were insisted upon, and I had to practice the piano from seven-thirty to eight every morning although the fires were not yet lit. My grandmother never allowed herself to sit in an armchair until the evening. Alcohol and tobacco were viewed with disfavor although stern convention compelled them to serve a little wine to guests. Only virtue was prized, virtue at the expense of intellect, health, happiness, and every mundane good.",p. 9,Bertrand Russell,Portraits from Memory and Other Essays (1956)
1986,"I was a solitary, shy, priggish youth. I had no experience of the social pleasures of boyhood and did not miss them. But I liked mathematics, and mathematics was suspect because it has no ethical content. I came also to disagree with the theological opinions of my family, and as I grew up I became increasingly interested in philosophy, of which they profoundly disapproved. Every time the subject came up they repeated with unfailing regularity, 'What is mind? No matter. What is matter? Never mind. ' After some fifty or sixty repetitions, this remark ceased to amuse me.",p. 9,Bertrand Russell,Portraits from Memory and Other Essays (1956)
...,...,...,...,...
27269,"Our language can be seen as an ancient city: a maze of little streets and squares, of old and new houses, and of houses with additions from various periods; and this surrounded by a multitude of new boroughs with straight regular streets and uniform houses.",§ 18,Ludwig Wittgenstein,Philosophical Investigations (1953)
2727

Let's check if the source have these pages, chapter, I,II and heading context have NaN. We will drop them too

In [106]:
print(df_new.isna().sum())

quote                 0
source             6770
author                0
heading_context    9115
dtype: int64


In [107]:

# ONLY_PUNCT_RE   = r'^[^\w\u4e00-\u9fff]+$'              # only punctuation/space
CONT_SPLIT_RE   = re.compile(r'\s*[:：]\s*\*+\s*')       # :*, ：*, : **, etc.

def clean_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = re.sub(r"\s+", " ", s).strip()
    return s.strip(" \t\n\r\"'，。；;:,-")

def pick_candidate(chunks):
    """Pick the longest non-empty, non-punct-only chunk."""
    cleaned = [clean_text(c) for c in chunks if isinstance(c, str)]
    cleaned = [c for c in cleaned if c and not re.fullmatch(ONLY_PUNCT_RE, c)]
    return max(cleaned, key=len) if cleaned else ""

# literals that are never meaningful as a full source *by themselves*
LITERAL_BAD_SOURCES = {
    ".", ",", "'.", "<",
    "Couplet", "Epilogue", "proverb",
    "Lennox", "Prospero", "Minotaur", "Cicero", "Björk", "Daniel",
}

# regexes for common “not meaningful” patterns (case-insensitive)
BAD_SOURCE_PATTERNS = [
    re.compile(r"^\s*scene\s*[ivxlcdm]+\s*$", re.I),           # Scene iv, Scene III, etc.
    re.compile(r"^\s*chapter\s*\d+\s*$", re.I),                # Chapter 1, Chapter 12
    re.compile(r"^\s*,?\s*p\.\s*(?:\d+|[ivxlcdm]+)(?:-\d+)?\s*$", re.I),  # p. 11, , p. xcv, p. 50-51
    re.compile(r"^\s*ibid\.?\s*$", re.I),                      # Ibid.
    re.compile(r"^\s*cited in\b.*$", re.I),                    # Cited in ...
    re.compile(r"^\s*\{\{cite.*$", re.I),                      # {{cite ...}}
    re.compile(ONLY_PUNCT_RE),                                 # only punctuation
    re.compile(r"^\s*[ivxlcdm]+\s*$", re.I),                   # bare Roman numeral: I, II, III...
    re.compile(r"^\s*\(2006\)\.\s*$", re.I),                   # (2006).
]

def source_is_meaningless(x) -> bool:
    """True if source is NaN OR one of the known 'meaningless' forms above."""
    if pd.isna(x):
        return True
    t = str(x).strip()
    if not t:
        return True
    if t in LITERAL_BAD_SOURCES:
        return True
    low = t.lower()
    if low in {s.lower() for s in LITERAL_BAD_SOURCES}:
        return True
    for pat in BAD_SOURCE_PATTERNS:
        if pat.fullmatch(t):
            return True
    return False

# --- build masks -----------------------------------------------------------

# "no heading context"
mask_no_context = df_new["heading_context"].isna()

# "no author" (strict NaN). If you also want to treat 'unknown' as missing, add .str.lower().eq('unknown')
mask_no_author  = df_new["author"].isna()

# source meaningless (includes NaN)
mask_bad_source = df_new["source"].apply(source_is_meaningless)

# rows to handle
mask_targets = mask_no_context & mask_no_author & mask_bad_source
targets = df_new[mask_targets]
print(f"Candidates to handle (bad source + no context/author): {len(targets)}")
# display(targets.head(20))  # uncomment to preview


Candidates to handle (bad source + no context/author): 0


In [114]:

cite_book_only = re.compile(r'^\s*\{\{\s*cite\s*book\b.*?\}\}\s*$', re.IGNORECASE | re.DOTALL)

# 2) Only an unclosed/truncated {{cite book ...}} tail
cite_book_truncated_only = re.compile(r'^\s*\{\{\s*cite\s*boo?k?\s*$', re.IGNORECASE)

mask_cite_only = df_new["source"].fillna("").str.match(cite_book_only)
mask_cite_trunc = df_new["source"].fillna("").str.match(cite_book_truncated_only)

rows_cite_only = df_new[mask_cite_only | mask_cite_trunc]

print(f"Rows where source is ONLY a cite book template (closed or truncated): {len(rows_cite_only)}")
display(rows_cite_only[["quote","author","source","heading_context"]])


Rows where source is ONLY a cite book template (closed or truncated): 0


,quote,author,source,heading_context


In [113]:
# setting them to NaN
df_new.loc[mask_cite_only | mask_cite_trunc, "source"] = np.nan


In [117]:
df_new.head(5)

,quote,source,author,heading_context
0,Un homme heureux est trop content du présent pour trop se soucier de l'avenir.,"A happy man is too satisfied with the present to dwell too much on the future. From ""Mes Projets d'Avenir"", a French essay written at age 18 for a school exam (18 September 1896). The Collected Papers of Albert Einstein Vol. 1 (1987) Doc. 22.",Albert Einstein,1890s
1,"Autoritätsdusel ist der größte Feind der Wahrheit. :: Another translation: Authority gone to one's head is the greatest enemy of truth. (Collected Papers, Volume 1, 1987): * Letter to Jost Winteler (July 8th, 1901), quoted in The Private Lives of Albert Einstein by Roger Highfields and Paul Carter (1993), p. 79. Einstein had been annoyed that Paul Drude, editor of Annalen der Physik, had dismissed some criticisms Einstein made of Drude's electron theory of metals.",Blind obedience to authority is the greatest enemy of truth.,Albert Einstein,1900s
2,"Lieber Habicht! / Es herrscht ein weihevolles Stillschweigen zwischen uns, so daß es mir fast wie eine sündige Entweihung vorkommt, wenn ich es jetzt durch ein wenig bedeutsames Gepappel unterbreche. . / Was machen Sie denn, Sie eingefrorener Walfisch, Sie getrocknetes, eingebüchstes Stück Seele. . ?","Dear Habicht, / Such a solemn air of silence has descended between us that I almost feel as if I am committing a sacrilege when I break it now with some inconsequential babble. . / What are you up to, you frozen whale, you smoked, dried, canned piece of soul. . ? Opening of a letter to his friend Conrad Habicht in which he describes his four revolutionary Annus Mirabilis papers (18 or 25 May 1905) Doc. 27",Albert Einstein,1900s
3,E=mc²,"The equation originally expressed the equivalence of mass and energym = L/c², which easily translates into the far more well-known E = mc² in Does the Inertia of a Body Depend Upon Its Energy Content? published in the Annalen der Physik (27 September 1905): ""If a body gives off the energy L in the form of radiation, its mass diminishes by L/c². "" In a later statement explaining the ideas expressed by this equation, Einstein summarized: ""It followed from the special theory of relativity that mass and energy are both but different manifestations of the same thing — a somewhat unfamiliar conception for the average mind. Furthermore, the equation E = mc², in which energy is put equal to mass, multiplied by the square of the velocity of light, showed that very small amounts of mass may be converted into a very large amount of energy and vice versa. The mass and energy were equivalent, according to the formula mentioned before. This was demonstrated by Cockcroft and Walton in 1932, experimentally. "" Atomic Physics (1948) by the J. Arthur Rank Organisation, Ltd. (Voice of A. Einstein. )",Albert Einstein,1900s
4,The mass of a body is a measure of its energy content.,"Ist die Trägheit eines Körpers von seinem Energieinhalt abhängig? (""Does the inertia of a body depend upon its energy content? "") Annalen der Physik 18, 639-641 (1905). Quoted in Concepts of Mass in Classical and Modern Physics by Max Jammer (1961), p. 177",Albert Einstein,1900s


In [118]:
df_new.to_csv("Preprocessed_quotes_dataframe.csv", index=False)
print("Saved:", "Preprocessed_quotes_dataframe.csv")

Saved: Preprocessed_quotes_dataframe.csv


In [120]:
df_2 = pd.read_csv("extracted_quotes_about.csv" , index_col = False)

In [121]:
df2_auth = (
    df_2
    .dropna(subset=["author"])  # only rows that actually have an author
    .drop_duplicates(subset=["quote", "source", "heading_context"], keep="first")
    [["quote", "source", "heading_context", "author"]]
    .rename(columns={"author": "author_new"})
)

# 2) Join onto df_new
merged = df_new.merge(
    df2_auth,
    on=["quote", "source", "heading_context"],
    how="left",
)

# 3) Preview planned changes
to_update = merged["author_new"].notna() & (merged["author"].fillna("") != merged["author_new"].fillna(""))
print(f"Rows to update: {to_update.sum()}")
display(merged.loc[to_update, ["quote", "source", "heading_context", "author", "author_new"]].head(20))


Rows to update: 5288


,quote,source,heading_context,author,author_new
464,"These days it is common knowledge that short waves are more powerful than long ones, as the very short ones, known as x-rays, damage living tissues. It took half-a-century to learn this fact: it was one of the great discoveries of young Albert Einstein of 1905. When he announced it leading researchers found it most incredible. .","Joseph Agassi, Radiation Theory and the Quantum Revolution (1993)",NaN,unknown,About Albert Einstein
465,". . do not be impressed by the imprint of a famous publishing house or the volumes of an author's publications. Bear in mind that Einstein needed only seventeen pages for his contribution which revolutionized physics, while there are graphomanics in asylums who use up mounds of paper every day.","Stanislav Andreski, The Social Sciences as Sorcery (1972, London: Deutsch), p 86",NaN,unknown,About Albert Einstein
466,"Paula Gunn Allen's description of the tribal culture is helpful in understanding this concept of energy dispersal: ""The closest analogy in Western thought is the Einsteinian understanding of matter as a special state or condition of energy. Yet even this concept falls short of the Native American understanding, for Einsteinian energy is essentially stupid, while energy in the Indian view is intelligence manifesting yet another way. ""","Bettina Aptheker Tapestries of Life: Women's Work, Women's Consciousness, and the Meaning of Daily Experience (1989)",NaN,unknown,About Albert Einstein
467,The astonishing thing about Einstein's equations is that they appear to have come out of nothing.,"Ernest Barnes, as quoted by Gerald James Whitrow, The Structure of the Universe: An Introduction to Cosmology (1949)",NaN,unknown,About Albert Einstein
468,"[During 1940s], Einstein was pursuing what he called his 'violon d'Ingres'—his uniﬁed ﬁeld theory. . The so-called strange particles were just being discovered, and the quantum theory was proving ever more powerful. Einstein simply was not much interested. His position was that it was useless to try to understand this new physics until the electron was understood. We now believe that understanding the electron is such an intimate part of the new physics that the electron cannot be understood by itself. But Besso took all his old friend's attempts extremely seriously, and Einstein gave him detailed explanations of his various formal manipulations. It was a dialogue that somehow reminds me of the plays of Samuel Beckett.","Jeremy Bernstein, Quantum Profiles, pp. 157-158.",NaN,unknown,About Albert Einstein
469,"I was particularly won over by his sweet disposition, by his general kindness, by his simplicity, and by his friendliness. Occasionally, gaiety would gain the upper hand and he would strike a more personal note and even disclose some detail of his day-to-day life. Then again, reverting to his characteristic mood of reflection and meditation, he would launch into a profound and original discussion of a variety of scientific and other problems. I shall always remember the enchantment of all those meetings, from which I carried away an indelible impression of Einstein's great human qualities.","Louis de Broglie, New Perspectives in Physics, p. 182",NaN,unknown,About Albert Einstein
470,"It is almost impertinent to talk of the ascent of man in the presence of two men, Newton and Einstein, who stride like gods. Of the two, Newton is the Old Testament god; it is Einstein who is the New Testament figure. He was full of humanity, pity, a sense of enormous sympathy. His vision of nature herself was that of a human being in the presence of something god-like, and that is what he always said about nature. He was fond of talking about God: 'God does not play at dice', 'God is not malicious'. Finally Niels Bohr one day said to him, 'Stop telling God what to do'. But that is not quite fair. Einstein was a man who could ask immensely simple questions. And what his life showed, and his work, is that when the

In [122]:
merged.loc[to_update, "author"] = merged.loc[to_update, "author_new"]
df_new_updated = merged.drop(columns=["author_new"]).reset_index(drop=True)

In [123]:
merged.loc[to_update, "author"] = merged.loc[to_update, "author_new"]
df_new_updated.shape

(29675, 4)

Let's also trim the extra starting and ending spaces and punctuations from the quotes before saving.

In [125]:
EDGE_CHARS = ' \t\r\n.,;:!?\'"“”‘’«»‹›()[]{}<>|/\\·•…`~--–—-'

def trim_edges(series: pd.Series) -> pd.Series:
    s = series.astype("object")               # keeps NaNs
    mask = s.notna()
    s.loc[mask] = (
        s.loc[mask].astype(str)
        .str.replace(r"\s+", " ", regex=True) 
        .str.strip(EDGE_CHARS)                
    )
    return s

df_new_updated["quote"]  = trim_edges(df_new_updated["quote"])



In [129]:
def parse_author(author):
    if pd.isna(author):
        return "canonical", "unknown"

    a = author.strip()
    lower = a.lower()

    if lower.startswith("about "):
        return "about", a[6:].strip()
    elif lower.startswith("misattributed to "):
        return "misattributed", a[17:].strip()
    elif lower.startswith("disputed with "):
        return "disputed", a[13:].strip()
    elif lower == "unknown":
        return "canonical", "unknown"
    else:
        return "canonical", a  # plain confirmed name

df_new_updated[["status", "target_name"]] = df_new_updated["author"].apply(
    lambda x: pd.Series(parse_author(x))
)

# now save with spread columns
df_new_updated.to_csv("quotes_spread.csv", index=False)

In [134]:
df_new_updated["quote"] = df_new_updated["quote"].astype(str).str.strip()
df_new_updated["source"] = df_new_updated["source"].astype(str).fillna("").str.strip()

# Lengths
q_len = df_new_updated["quote"].str.len()
s_len = df_new_updated["source"].str.len()

# 1) Drop long quotes
removed_quotes = (q_len > 1000).sum()
df_new = df_new_updated[q_len <= 1000].copy()

# 2) Keep row, but cap source to 1000 chars (or 300 if you prefer)
df_new["source"] = df_new_updated["source"].str.slice(0, 1000)

print(f"Removed {removed_quotes} quotes >1000 chars; kept {len(df_new)} rows.")

Removed 1813 quotes >1000 chars; kept 27862 rows.


In [135]:
df_new.shape

(27862, 6)

In [137]:
df_new.to_csv("quotes_database_final.csv", index=False)

In [126]:
df_new_updated.to_csv("Preprocessed_quotes_dataframe_updated_1.csv", index=False)
print("Saved:", "Preprocessed_quotes_dataframe_updated_1.csv")

Saved: Preprocessed_quotes_dataframe_updated_1.csv


In [138]:
pip install fastapi



   ---------------------------------------- 0/2 [starlette]
   ---------------------------------------- 0/2 [starlette]
   ---------------------------------------- 0/2 [starlette]
   -------------------- ------------------- 1/2 [fastapi]
   -------------------- ------------------- 1/2 [fastapi]
   -------------------- ------------------- 1/2 [fastapi]
   -------------------- ------------------- 1/2 [fastapi]
   ---------------------------------------- 2/2 [fastapi]



In [139]:
pip install uvicorn 

In [140]:
pip install neo4j

What I did above is that, first time I extracted the information frOm the dump when the heading was quote about I set the author as unknown. I reextracted the information after changing the extractor little bit. This time I set the author as about title when the heading is quote about or quote for. And then merged replaced in the new dataframe.

* Now the preprocessing is almost done. The only thing I have to decide is the len of the quote and source.
* And the nodes and the properties of the graph datbase.